In [1]:
import openai
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import json

In [2]:
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY")
)

In [3]:
def get_current_weather(location):
     
     url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

     querystring = {"text":"location","language":"en"}

     headers = {
	 "X-RapidAPI-Key": "66e16bab9emsh9ac9ebfdc2f54dbp13176bjsn2693206354cd",
	 "X-RapidAPI-Host": "ai-weather-by-meteosource.p.rapidapi.com"
     }

     response = requests.get(url, headers=headers, params=querystring)

     print(type(response))
     return response.json()

In [82]:
functions =[
             
                    {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        
     ] 

In [84]:
user_message="what is the temperature of jaipur?"
messages=[]
messages=[
    {
        "role": "system", "content": "You are an weather assistant, which gives weather data"},
        {"role": "user", "content": user_message

    }]
#messages.append({"role": "user", "content":user_message})
completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
  messages=  messages,
  functions= functions,
  max_tokens= 40
)

In [85]:
completion

ChatCompletion(id='chatcmpl-9LiMLBfCmWXXBkmudIlELPG0R0lWk', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Jaipur","unit":"celsius"}', name='get_current_weather'), tool_calls=None))], created=1714962433, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=21, prompt_tokens=93, total_tokens=114))

In [86]:
response=completion.choices[0].message
response


ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Jaipur","unit":"celsius"}', name='get_current_weather'), tool_calls=None)

In [92]:
function_name = response.function_call.name
print(function_name)  # Output: get_current_weather



get_current_weather


In [93]:
import ast
location = ast.literal_eval(response.function_call.arguments)['location']
print(location)

Jaipur


In [94]:
messages.append(response)
messages.append(
    {
    "role":"function",
    "name":"function_name",
    "content":"location"
    }
)

In [95]:
messages

[{'role': 'system',
  'content': 'You are an weather assistant, which gives weather data'},
 {'role': 'user', 'content': 'what is the temperature of jaipur?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Jaipur","unit":"celsius"}', name='get_current_weather'), tool_calls=None),
 {'role': 'function', 'name': 'function_name', 'content': 'location'}]

In [100]:
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    functions=functions    
    )

In [1]:
print(second_response.choices[0].message)


NameError: name 'second_response' is not defined

In [102]:
second_response

ChatCompletion(id='chatcmpl-9LiSgbUrCcjaaM7ca5lqUFkzjAz07', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in Jaipur is 33°C.', role='assistant', function_call=None, tool_calls=None))], created=1714962826, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3b956da36b', usage=CompletionUsage(completion_tokens=12, prompt_tokens=123, total_tokens=135))